In [ ]:
import numpy as np
import larcv 
import h5py
import matplotlib.pyplot as plt

In [ ]:
# Create the config builder and turn verbosity down low:
from larcv.config_builder import ConfigBuilder
cb = ConfigBuilder()
cb.set_parameter(["/home/giorgio/merged_sample_0.h5"], "InputFiles") # Pass input files as a list because you can combine them here
cb.set_parameter(5, "ProcessDriver", "IOManager", "Verbosity")
cb.set_parameter(5, "ProcessDriver", "Verbosity")
cb.set_parameter(5, "Verbosity")

In [ ]:
name = "demo"

# Bring in the wires:
cb.add_batch_filler(
    datatype  = "sparse3d",
    producer  = "dunevoxels",
    name      = name+"data", # This is the name in the output dict to place this data
    MaxVoxels = 1000, # Zero pad the empty voxels for this many voxels
    Augment   = False, # Apply on-the-fly augmentation
    Channels  = [4,] # How many projection IDs?
)

# Bring in the labels:
cb.add_batch_filler(
    datatype  = "particle",
    producer  = "all",
    name      = name+"label",
)


# Build up the data_keys:
data_keys = {
    'image': name + 'data',
    'label': name + 'label'
    }

import json
print(json.dumps(cb.get_config(), indent=2))

In [ ]:
from larcv.queueloader import queue_interface
# from larcv.distributed_queue_interface import queue_interface # This is the multi-node, parallel version!

queue_interface = queue_interface(random_access_mode="random_blocks", seed=1234)
queue_interface.no_warnings()

# This gets the queue interface started:
batch_size=1


# Prepare data managers: (You can have more than one!  Train / test / anything)
io_config = {
    'filler_name' : name,
    'filler_cfg'  : cb.get_config(),
    'verbosity'   : 5,
    'make_copy'   : False # This is if you want to make a copy of data in python
}


queue_interface.prepare_manager(name, io_config, batch_size, data_keys, color=None)

In [ ]:
data_dict = queue_interface.fetch_minibatch_data(name, pop=True,fetch_meta_data=True)

In [ ]:
print(data_dict.keys())

In [ ]:
# Print the vertex locations of this data:
print(data_dict['image'])

In [ ]:
plt.imshow(data_dict["image"][0])

In [ ]:
queue_interface.prepare_next(name) # prepare more data for the dataset tagged with `name`

In [ ]:
re_data_dict = queue_interface.fetch_minibatch_data(name, pop=False,fetch_meta_data=True)
print(re_data_dict["label"]["_pdg"])

In [ ]:
# Calling with pop = True will discard this data after you use it - so you get it one more time,
# but the next time is different.  In general, you usually want pop = True.
re_data_dict = queue_interface.fetch_minibatch_data(name, pop=True,fetch_meta_data=True)
queue_interface.prepare_next(name)
print(re_data_dict['label']["_vtx"])

In [ ]:
re_data_dict = queue_interface.fetch_minibatch_data(name, pop=True,fetch_meta_data=True)
queue_interface.prepare_next(name)
print(re_data_dict['label']["_vtx"])